In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "flax-community/t5-recipe-generation"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [10]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [11]:
def predict_recipe(title_ingredients):
    """
    Generate recipe directions from title and ingredients.
    Format expected by the model: 'Title: <title> Ingredients: <comma-separated ingredients>'
    """
    # Correct format
    text = f"Title: {title_ingredients['title']} Ingredients: {title_ingredients['ingredients']}"
    
    # Tokenize and generate
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)
    
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=200,
        num_beams=5,
        early_stopping=True
    )
    
    # Decode and return
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    start = decoded.find("directions")
    return decoded[start:] if start != -1 else decoded

In [12]:
predict_recipe({"title" : "Smoothie",
                "ingredients" :"Watermelon, Kiwi, Apple and Frozen Banana Smoothie"})

'directions: place all ingredients in a blender. blend until smooth. serve immediately.'